# Modelo de Clasificación Local de Extremo a Extremo

Este notebook implementa un pipeline completo de Machine Learning de forma local:

1.  **Carga de Datos**: Usa un dataset de texto real (`ag_news`) de la librería `datasets`.
2.  **Análisis y Tokenización**: Analiza la longitud de los textos usando un tokenizador de BERT.
3.  **Generación de Embeddings**: Convierte el texto en vectores numéricos (embeddings) usando un modelo BERT pre-entrenado.
4.  **Entrenamiento**: Entrena un clasificador XGBoost con los embeddings generados.
5.  **Evaluación**: Evalúa el rendimiento del modelo final.

Todo el proceso se ejecuta localmente sin dependencias de la nube.

## 1. Instalación y Configuración

In [ ]:
#!pip install transformers torch datasets scikit-learn xgboost pandas seaborn matplotlib tqdm

In [ ]:
# --- CONFIGURACIÓN GENERAL ---
import pandas as pd
import numpy as np
import os
import pickle
import unicodedata
import seaborn as sns
import matplotlib.pyplot as plt
import time
import torch
import xgboost as xgb
from tqdm.auto import tqdm

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# --- Parámetros de Configuración ---
BERT_MODEL_NAME = 'bert-base-uncased'
MAX_SAMPLES = 2500 # Limitar el número de muestras para que la ejecución sea más rápida. Poner a None para usar el dataset completo.
MAX_TOKEN_LENGTH = 128 # Max longitud para truncar/rellenar tokens.

# --- Configuración de Dispositivo (GPU o CPU) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# --- Definición de Rutas Locales ---
job_id = f"local-bert-job-{int(time.time())}"
BASE_DIR = "datos_locales"
INPUT_DIR = os.path.join(BASE_DIR, "input")
PROCESSED_DIR = os.path.join(BASE_DIR, "processed", job_id)
MODEL_OUTPUT_DIR = os.path.join(BASE_DIR, "model_output", job_id)

os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)
os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)

INPUT_EMBEDDINGS_FILENAME = "text_embeddings.csv"
LOCAL_EMBEDDINGS_PATH = os.path.join(INPUT_DIR, INPUT_EMBEDDINGS_FILENAME)

print(f"\nID de trabajo para esta ejecución: {job_id}")
print(f"Ruta para embeddings generados: {LOCAL_EMBEDDINGS_PATH}")

## 2. Carga, Análisis y Tokenización de Datos

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from tqdm.auto import tqdm
from transformers import AutoTokenizer
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

print("Cargando dataset '20 Newsgroups' desde Scikit-learn (100% offline)...")
# NOTA: La primera vez que se ejecute, scikit-learn puede descargar y guardar los datos en caché.
# Después de eso, siempre se cargará localmente.

# Cargamos tanto el conjunto de entrenamiento como el de prueba para tener más datos
try:
    # El parámetro 'remove' limpia los metadatos para que el modelo se centre en el contenido del texto.
    newsgroups_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))
    newsgroups_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))
except Exception as e:
    print(f"\nERROR: No se pudo cargar el dataset '20 Newsgroups'. Causa: {e}")
    print("Si es un error de red, ejecuta este notebook una vez en una máquina con internet para que se descargue y guarde en caché.")
    raise

# Combinamos los datos en un solo DataFrame de pandas
all_text = newsgroups_train.data + newsgroups_test.data
all_targets = list(newsgroups_train.target) + list(newsgroups_test.target)
label_names = newsgroups_train.target_names
all_label_names = [label_names[i] for i in all_targets]

df = pd.DataFrame({
    'text': all_text,
    'label_name': all_label_names
})

# Tomar una muestra si se especificó para acelerar el proceso
if 'MAX_SAMPLES' in locals() and MAX_SAMPLES is not None:
    print(f"Tomando una muestra aleatoria de {MAX_SAMPLES} registros.")
    df = df.sample(n=MAX_SAMPLES, random_state=42).reset_index(drop=True)

print(f"Dataset cargado con {len(df)} filas.")
print("\nPrimeras 5 filas:")
print(df.head())

print("\nDistribución de clases (en la muestra):")
print(df['label_name'].value_counts())

# Cargar tokenizador de BERT (esto todavía necesita internet la primera vez que se ejecuta)
print(f"\nCargando tokenizador: {BERT_MODEL_NAME}")
try:
    tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
except (ConnectionError, OSError) as e:
    print(f"\nERROR: No se pudo descargar el tokenizador. Causa: {e}")
    print("Si el problema es de red, descarga la carpeta del modelo 'bert-base-uncased' manualmente desde el Hub y cárgalo desde la ruta local.")
    raise

# Medir longitud de tokens
print("Analizando longitud de los textos en tokens...")
# Usamos tqdm para ver una barra de progreso, ya que puede tardar un poco
df['token_length'] = [len(tokenizer.encode(text, max_length=512, truncation=True)) for text in tqdm(df['text'])]

# Visualizar la distribución de la longitud de tokens
plt.figure(figsize=(10, 6))
sns.histplot(df['token_length'], bins=50, kde=True)
plt.title('Distribución de la Longitud de Tokens por Texto')
plt.xlabel('Longitud de Tokens')
plt.ylabel('Frecuencia')
# Usamos MAX_TOKEN_LENGTH definido en la primera celda
plt.axvline(x=MAX_TOKEN_LENGTH, color='r', linestyle='--', label=f'Max Length = {MAX_TOKEN_LENGTH}')
plt.legend()
plt.show()

print(f"Longitud promedio de tokens: {df['token_length'].mean():.2f}")

## 3. Generación de Embeddings con BERT
Este es el paso más intensivo computacionalmente. Se recomienda usar una GPU.

In [ ]:
print(f"Cargando modelo pre-entrenado: {BERT_MODEL_NAME}")
model = AutoModel.from_pretrained(BERT_MODEL_NAME).to(device)
model.eval() # Poner el modelo en modo de evaluación

def get_bert_embeddings(batch_text):
    """Tokeniza un lote de texto y obtiene el embedding [CLS] de BERT."""
    inputs = tokenizer(batch_text, padding=True, truncation=True, 
                       max_length=MAX_TOKEN_LENGTH, return_tensors='pt')
    
    # Mover tensores al dispositivo (GPU/CPU)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Usamos el embedding del token [CLS] (índice 0)
    cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return cls_embeddings

print("\nGenerando embeddings... Esto puede tardar varios minutos.")
batch_size = 32
all_embeddings = []

# tqdm ofrece una barra de progreso
for i in tqdm(range(0, len(df), batch_size)):
    batch_df = df.iloc[i:i+batch_size]
    batch_text = batch_df['text'].tolist()
    embeddings = get_bert_embeddings(batch_text)
    all_embeddings.append(embeddings)

# Combinar todos los embeddings de los lotes
final_embeddings = np.vstack(all_embeddings)

# Crear un DataFrame con los embeddings
embedding_cols = [f'dim_{i}' for i in range(final_embeddings.shape[1])]
df_embeddings = pd.DataFrame(final_embeddings, columns=embedding_cols)

# Unir los embeddings con las etiquetas originales
# Usamos 'label_name' como la etiqueta de texto que queremos predecir
df_final = pd.concat([df[['label_name']].rename(columns={'label_name': 'label'}), df_embeddings], axis=1)

# Guardar el resultado para los siguientes pasos
df_final.to_csv(LOCAL_EMBEDDINGS_PATH, index=False)

print(f"\n✓ Embeddings generados y guardados en: {LOCAL_EMBEDDINGS_PATH}")
print("Dimensiones del DataFrame final:", df_final.shape)
print(df_final.head())

## 4. División y Codificación de Datos

In [ ]:
print("\n--- Ejecutando Lógica de División y Codificación ---")

# 1. Cargar los datos de entrada (embeddings generados)
print(f"Cargando datos desde {LOCAL_EMBEDDINGS_PATH}")
df = pd.read_csv(LOCAL_EMBEDDINGS_PATH)
print(f"Datos cargados. {len(df)} filas.")

# 2. Codificar las etiquetas
label_col_name = 'label'
encoded_label_col = f"{label_col_name}_encoded"
embedding_cols = [col for col in df.columns if col.startswith('dim_')]

print(f"Columnas de embedding detectadas: {len(embedding_cols)}")
print(f"Columna de etiquetas: {label_col_name}")

# La lógica de filtrar clases con <10 instancias se mantiene por robustez
label_counts = df[label_col_name].value_counts()
valid_labels = label_counts[label_counts > 9].index
df = df[df[label_col_name].isin(valid_labels)].reset_index(drop=True)

print(f"Filtrado completado. Se conservaron {len(valid_labels)} clases con más de 9 muestras.")

print("Codificando etiquetas...")
label_encoder = LabelEncoder()
df[encoded_label_col] = label_encoder.fit_transform(df[label_col_name])
num_classes = len(label_encoder.classes_)
print(f"Se detectaron y codificaron {num_classes} clases.")
print("Mapeo de etiquetas:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

X = df[embedding_cols].values
y = df[encoded_label_col].values

# 3. Dividir los datos
print("Dividiendo los datos en conjuntos de entrenamiento, validación y prueba...")
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, random_state=42, stratify=y_trainval)

# 4. Guardar los conjuntos de datos en archivos locales
test_features_path = os.path.join(PROCESSED_DIR, "test_features.csv")
test_labels_path = os.path.join(PROCESSED_DIR, "test_labels.csv")
pd.DataFrame(X_test, columns=embedding_cols).to_csv(test_features_path, index=False)
pd.DataFrame(y_test, columns=[encoded_label_col]).to_csv(test_labels_path, index=False)

# Guardar el codificador de etiquetas
label_encoder_path = os.path.join(PROCESSED_DIR, "label_encoder.pkl")
with open(label_encoder_path, "wb") as f:
    pickle.dump(label_encoder, f)

print(f"\n✓ Procesamiento completado exitosamente!")

## 5. Entrenamiento del Modelo (Local con XGBoost)

In [ ]:
print("\n--- Ejecutando Entrenamiento del Modelo Local con XGBoost (GPU) ---\n")
print("--- PASO 1: Búsqueda de Hiperparámetros con GridSearchCV (SIN Parada Temprana) ---")

# Modelo XGBoost configurado para usar GPU
xgb_model = xgb.XGBClassifier(
    objective="multi:softprob",
    num_class=num_classes,
    eval_metric="mlogloss",
    seed=42,
    use_label_encoder=False,
    tree_method='hist',  # Método de construcción de árboles compatible con GPU
    device='cuda'        # Especifica usar GPU (XGBoost 2.0+)
    # Para versiones anteriores usar: gpu_id=0, tree_method='gpu_hist'
)

# La parrilla de hiperparámetros
param_grid = {
    'learning_rate': [0.05, 0.1],
    'max_depth': [4, 6],
    'subsample': [0.8, 1.0],
    'n_estimators': [100, 150]
}

# Configuración de GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='neg_log_loss',
    n_jobs=1,  # Cambiado a 1 para GPU (evita conflictos de paralelización)
    cv=3,
    verbose=2
)

# Lanzamos la búsqueda
grid_search.fit(X_train, y_train)

print("\nBúsqueda de hiperparámetros completada.")
print(f"Mejores parámetros encontrados: {grid_search.best_params_}")
print(f"Mejor score (neg_log_loss) en validación cruzada: {grid_search.best_score_:.4f}")

print("\n--- PASO 2: Entrenamiento del Modelo Final con los Mejores Parámetros y Parada Temprana ---")

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_

# Modelo final con GPU
final_model = xgb.XGBClassifier(
    objective="multi:softprob",
    num_class=num_classes,
    eval_metric="mlogloss",
    seed=42,
    use_label_encoder=False,
    tree_method='hist',  # Método compatible con GPU
    device='cuda',       # GPU habilitada
    **best_params
)

# Entrenamiento con parada temprana
final_model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=10,
    verbose=False
)

print("\nEntrenamiento del modelo final completado.")

# Guardamos el modelo
best_model = final_model
LOCAL_MODEL_PATH = os.path.join(MODEL_OUTPUT_DIR, "modelo_xgboost_gpu.pkl")

with open(LOCAL_MODEL_PATH, 'wb') as f:
    pickle.dump(best_model, f)

print(f"\n✓ Mejor modelo guardado en: {LOCAL_MODEL_PATH}")

# Opcional: Verificar si GPU está siendo utilizada
import subprocess
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    if result.returncode == 0:
        print("\n--- Estado de GPU ---")
        print("GPU disponible y siendo utilizada por XGBoost")
    else:
        print("\nAdvertencia: No se pudo verificar el estado de GPU")
except FileNotFoundError:
    print("\nAdvertencia: nvidia-smi no encontrado, verifica la instalación de CUDA")

## 6. Evaluación del Modelo (Local)

In [ ]:
print("\n--- Ejecutando Evaluación del Modelo en el Conjunto de Prueba ---")

# 1. Cargar los artefactos de prueba
X_test_eval = pd.read_csv(test_features_path).values
y_test_eval = pd.read_csv(test_labels_path).values.flatten()

with open(label_encoder_path, 'rb') as f:
    label_encoder_eval = pickle.load(f)

with open(LOCAL_MODEL_PATH, 'rb') as f:
    loaded_model = pickle.load(f)

print(f"Modelo y datos de prueba cargados: {X_test_eval.shape[0]} muestras.")

# 2. Obtener predicciones
y_pred_eval = loaded_model.predict(X_test_eval)

# 3. Calcular y mostrar métricas
print("\n--- Resultados de la Evaluación ---")
acc = accuracy_score(y_test_eval, y_pred_eval)
report = classification_report(y_test_eval, y_pred_eval, target_names=label_encoder_eval.classes_)
cm = confusion_matrix(y_test_eval, y_pred_eval)

print(f"Accuracy: {acc:.4f}\n")
print("Reporte de Clasificación:")
print(report)

print("\nMatriz de Confusión:")
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=label_encoder_eval.classes_, yticklabels=label_encoder_eval.classes_)
plt.ylabel('Etiqueta Real')
plt.xlabel('Etiqueta Predicha')
plt.title('Matriz de Confusión')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("\n--- Evaluación Completada ---")